- 画像分類の事前学習済みモデルはすべて224*224画素の画像で学習済

# ファインチューニングと転移学習
- ファインチューニングは、学習済モデルのパラメタを初期値として利用するが、すべてのレイヤーで再学習する
    - 学習データが大量にある場合に向いている → CIFAR10は5万件もあるので本章ではファインチューニングを用いる
- 転移学習は、学習済モデルのパラメタのうち、入力に近い部分は固定し、出力に近い部分のみ学習する
    - 学習データが少ない場合に向いている

In [1]:
# 必要ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from IPython.display import display

In [2]:
# PyTorch関連ライブラリのインポート

import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets

In [3]:
# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=5)

In [4]:
# GPUチェック

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
# 著者の共通関数読み込み
import sys
sys.path.append("../../lib/")
from pythonlibs.torch_lib1 import *

# 11.4 nn.AdaptiveAvgPool2d

- AdaptiveXXXPool2d で指定するパラメタは、変換後の画素数
- 集約化の方法はXXXで決まる

In [6]:
p = nn.AdaptiveAvgPool2d((1, 1))
print(p)

l1 = nn.Linear(32, 10)
print(l1)

AdaptiveAvgPool2d(output_size=(1, 1))
Linear(in_features=32, out_features=10, bias=True)


In [7]:
# 事前学習済みモデルのシミュレーション
# 100枚 * 32チャネル * 16画素 * 16画素
inputs = torch.randn(100, 32, 16, 16)
m1 = p(inputs)
m2 = m1.view(m1.shape[0], -1)
m3 = l1(m2)

# チャネルごとに平均値の1画素のみとなる
print(m1.shape)
# 画像ごとに一次元ベクトル化
print(m2.shape)
print(m3.shape)

torch.Size([100, 32, 1, 1])
torch.Size([100, 32])
torch.Size([100, 10])


- この仕組みで、inputsのshapeが[100, 32, 8, 8]でも[100, 32, 4, 4]でも、同じ線形関数 Linear(32, 10) で受け取れる